In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

class YahooFinanceData:
    def __init__(self, ticker, start_date, end_date, interval='1d'):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval
        self.data = None
    
    def fetch_data(self):
        """
        Fetches historical data from Yahoo Finance based on the initialized parameters.
        """
        try:
            self.data = yf.download(
                self.ticker, 
                start=self.start_date, 
                end=self.end_date, 
                interval=self.interval
            )
            print(f"Data fetched successfully for {self.ticker}")
        except Exception as e:
            print(f"An error occurred while fetching data: {e}")
    
    def get_data(self):
        """
        Returns the fetched data.
        """
        if self.data is not None:
            return self.data
        else:
            print("Data not available. Please fetch data first.")
            return None

# Define your indicator classes as per the previous examples (BollingerBands, EMA, etc.)
# ...

class Backtest:
    def __init__(self, strategy, data, initial_balance=10000):
        self.strategy = strategy
        self.data = data
        self.initial_balance = initial_balance
        self.balance = initial_balance
        self.position = 0  # 0 means no position, 1 means long position
        self.history = []

    def execute(self):
        buy_signals, sell_signals = self.strategy.generate_signals()
        close_prices = self.data['Close']

        for i in range(len(close_prices)):
            if buy_signals.iloc[i] and self.position == 0:
                # Buy action
                self.position = 1
                buy_price = close_prices.iloc[i]
                self.history.append({'Date': close_prices.index[i], 'Action': 'Buy', 'Price': buy_price})
                print(f"Buy at {buy_price} on {close_prices.index[i]}")
            elif sell_signals.iloc[i] and self.position == 1:
                # Sell action
                self.position = 0
                sell_price = close_prices.iloc[i]
                profit = (sell_price - buy_price) / buy_price * self.initial_balance
                self.balance += profit
                self.history.append({'Date': close_prices.index[i], 'Action': 'Sell', 'Price': sell_price, 'Profit': profit})
                print(f"Sell at {sell_price} on {close_prices.index[i]}; Profit: {profit}")

        print(f"Final balance: {self.balance}")
        return pd.DataFrame(self.history)

# Example usage:
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2021-01-01"
interval = "1d"

yahoo_data = YahooFinanceData(ticker, start_date, end_date, interval)
yahoo_data.fetch_data()
data = yahoo_data.get_data()

if data is not None:
    # Initialize your indicators with the fetched data
    vi = VortexIndicator(data)
    ema = EMA(data, short_window=5, long_window=10)
    rsi = RSI(data)

    # Combine indicators into a strategy
    strategy = CombinedIndicatorStrategy([vi, ema, rsi])
    
    # Run the backtest
    backtest = Backtest(strategy, data)
    backtest_history = backtest.execute()
    print(backtest_history)
